In [1]:
pip install splinter webdriver_manager


Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import Splinter, BeautifulSoup, and Pandas and other dependencies
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import pymongo
import requests
import pathlib
import pprint
import time

# Web Scraping

In [2]:
# Set the executable path and initialize the chrome browser in splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\jabuk\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


## Visit the NASA mars news site

In [3]:
# Visit the mars nasa news site
url_nasa = 'https://mars.nasa.gov/news/'

# Optional delay for loading the page
browser.visit(url_nasa)

In [4]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
# print(soup.prettify())

In [5]:
# Scrape latest news title
# Latest news title as of today, 4/9/2021: "NASA's Mars helicopter to make first flight attempt Sunday"
latest_news = soup.findAll('div', class_="content_title")
news_title= latest_news[1].text
news_title

"NASA's Mars Helicopter to Make First Flight Attempt Sunday"

In [6]:
# Scrape Latest news description text
descriptions = soup.findAll('div', class_="article_teaser_body")
news_desc = descriptions[0].text
news_desc

'The small rotorcraft’s “Wright brothers moment” is two Mars days away.'

## JPL Space Images Featured Image

In [7]:
# Visit JPL space images Mars URL to find the featured 
url_jpl = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(url_jpl)

In [8]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

In [9]:
#Scrape relative image URL of the featured image from the jpl site using soup
images = soup.findAll('img', class_="headerimage fade-in")
featured_img= images[0].attrs['src']
featured_img

'image/featured/mars1.jpg'

In [10]:
# Featured image URL
featured_img_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'+featured_img
featured_img_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg'

## Mars Facts

In [12]:
# Visit space-facts.com 
url_mars = 'https://space-facts.com/mars/'
browser.visit(url_mars)

MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=61811): Max retries exceeded with url: /session/990c01e5806f1114f8c97a277f3de7b7/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000025921E09308>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))

In [ ]:
# Reading html
rawdata_mars = pd.read_html(url_mars)
rawdata_mars

#### Data Cleaning Stage

In [ ]:
# Converting html results to a dataframe and selecting table with Mars Facts
temp_mars_df = rawdata_mars[1]
temp_mars_df

In [ ]:
# Rename column Mars - Earth Comparison to Attributes
mars_df = temp_mars_df.rename(columns= {'Mars - Earth Comparison': 'Attributes'})
mars_df

In [7]:
# Dropped the column wit Earth Facts 
mars_df= mars_df.drop(columns = ["Earth"])
mars_df

,Attributes,Mars
0,Diameter:,"6,779 km"
1,Mass:,6.39 × 10^23 kg
2,Moons:,2
3,Distance from Sun:,"227,943,824 km"
4,Length of Year:,687 Earth days
5,Temperature:,-87 to -5 °C


In [8]:
# Export data to HTML
mars_df.to_html("mars_data")

## Mars Hemispheres

In [3]:
# Visit the USGS astrogeology page for hemisphere data from Mars
base_url ="https://astrogeology.usgs.gov"
url_hemisphere = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url_hemisphere)

In [4]:
# Extract the .html out of the browser & create beautifulsoup object
hem_html = browser.html
hem_soup = BeautifulSoup(hem_html,'html.parser')
print(hem_soup.prettify())

<html lang="en">
 <head>
  <link href="//ajax.googleapis.com/ajax/libs/jqueryui/1.11.3/themes/smoothness/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <title>
   Astropedia Search Results | USGS Astrogeology Science Center
  </title>
  <meta content="USGS Astrogeology Science Center Astropedia search results." name="description"/>
  <meta content="USGS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
  <meta content="x61hXXVj7wtfBSNOPnTftajMsZ5yB2W-qRoyr7GtOKM" name="google-site-verification"/>
  <!--<link rel="stylesheet" href="http://fonts.googleapis.com/css?family=Open+Sans:400italic,400,bold"/>-->
  <link href="/css/main.css" media="screen" rel="stylesheet"/>
  <link href="/css/print.css" media="print" rel="styles

In [5]:
# Finds images in divs with class=item
results = hem_soup.find_all('div', class_='item')
results

[<div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a><div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div> <!-- end description --></div>,
 <div class="item"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="/cache/images/08eac6e22c07fb1fe72223a79252de20_schiapa

In [23]:
# Empty list to store key-value pairs (dictionary data) 
hem_img_urls = []

# Loop through each hemisphere 
# Click on link to find image url of the enhanced resolution
for hem_img in results:
    hem_dict = {}
    
    href = hem_img.find('a', class_='itemLink product-item')
    link = base_url + href['href']
    browser.visit(link)
    
    time.sleep(1)
    
    hem_html2 = browser.html
    hem_soup2 = BeautifulSoup(hem_html2, 'html.parser')
    
    img_title = hem_soup2.find('div', class_='content').find('h2', class_='title').text
    hem_dict['title'] = img_title
    
    img_url = hem_soup2.find('div', class_='downloads').find('a')['href']
    hem_dict['url_img'] = img_url
    
    # Append dictionary to list
    hem_img_urls.append(hem_dict)
      
hem_img_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'url_img': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'url_img': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'url_img': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'url_img': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [24]:
# Quit browser when finished
browser.quit()